In [1]:
#Include Libraries
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

### Configuration of Neural Network

In [2]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

## Step 1 Load Data  

In [4]:
print("Load DataSet")
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("/tmp/data/", one_hot = True)
print()

print("Total 70,000 Images")
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Validation-set:\t{}".format(len(data.test.labels)))
print("- Test-set:\t\t{}".format(len(data.validation.labels)))
print()

print("Properties of image")
# The number of pixels in each dimension of an image.
img_size = 28
print("Image_Size: ",img_size)

# The images are stored in one-dimensional arrays of this length.
img_size_flat = 28*28
print("img_size_flat: ",img_size_flat)

# Tuple with height and width of images used to reshape arrays.
img_shape = (28,28)
print("img_shape: ",img_shape)

# Number of classes, one class for each of 10 digits.
num_classes = 10
print("num_classes: ",num_classes)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1
print("num_channels: ",num_channels)

Load DataSet
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz

Total 70,000 Images
Size of:
- Training-set:		55000
- Validation-set:	10000
- Test-set:		5000

Properties of image
Image_Size:  28
img_size_flat:  784
img_shape:  (28, 28)
num_classes:  10
num_channels:  1


## Step 2  Tensorflow Setup Part A Tensorflow Graph

A TensorFlow graph consists of the following parts which will be detailed below: 

* Placeholder variables used for inputting data to the graph.
* Variables that are going to be optimized so as to make the convolutional network perform better.
* The mathematical formulas for the convolutional network.
* A cost measure that can be used to guide the optimization of the variables.
* An optimization method which updates the variables.

In [5]:
#Helper Functions

#1. Construct Variables Wg and Bias
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

#2. Create a Convolutional Layer
def new_convo_layer(input, num_input_channels, filter_size, num_filters, use_pooling = True):
    
    #filter_size, Width and height of each filter.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters] 
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    
    #Layer
    # Strides = [1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image 
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME') 
    layer = layer + biases
    
    #We use Pooling to down-sample the image resolution 
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)
    return layer, weights

#3. Flattening a layer
def flatten_layer(layer):
    # Get the shape of the input layer.
    # layer_shape == [num_images, img_height, img_width, num_channels]
    layer_shape = layer.get_shape()
    
    # The number of features is: img_height * img_width * num_channels
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features
    

#4. New FUlly Connected Layer
def new_fc_layer(input, num_inputs, num_outputs, use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer
    
    

### A.1 Placeholder

Placeholder variables serve as the input to the TensorFlow computational graph

In [6]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
#The convolutional layers expect x to be encoded as a 4-dim tensor
# -1: Num_Images, img_size == img_H == Img_W, Num_Channels
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

y_true_cls = tf.argmax(y_true, axis=1)

### A.2 CNN Model

In [8]:
#1. Convolutional Layer 1
layer_conv1, weights_conv1 = new_convo_layer(input=x_image, 
                                            num_input_channels=num_channels,
                                            filter_size=filter_size1,
                                            num_filters=num_filters1,
                                            use_pooling=True)
print("Layer_Convo_1")
print(layer_conv1)

#2. Convolutional Layer 2
layer_conv2, weights_conv2 = new_convo_layer(input=layer_conv1, 
                                            num_input_channels=num_filters1,
                                            filter_size=filter_size2,
                                            num_filters=num_filters2,
                                            use_pooling=True)
print("Layer_Convo_2")
print(layer_conv2)

#3. Flatten Layer
layer_flat, num_features = flatten_layer(layer_conv2)

print("layer_flat")
print(layer_flat)

print("num_features")
print(num_features)

#4. Fully Connected Layer 1
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)
print("layer_fc1")
print(layer_fc1)

#5. Fully Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)
print("layer_fc2")
print(layer_fc2)

#Predicted Class
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)

Layer_Convo_1
Tensor("Relu_3:0", shape=(?, 14, 14, 16), dtype=float32)
Layer_Convo_2
Tensor("Relu_4:0", shape=(?, 7, 7, 36), dtype=float32)
layer_flat
Tensor("Reshape_2:0", shape=(?, 1764), dtype=float32)
num_features
1764
layer_fc1
Tensor("Relu_5:0", shape=(?, 128), dtype=float32)
layer_fc2
Tensor("add_7:0", shape=(?, 10), dtype=float32)


### A.3 Cost-function to be optimized

In [10]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

### A.4 Optimization method and Performance measures

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Step 2 Tensorflow Run Part B TensorFlow Session


In [12]:
session = tf.Session()
#sess = tf.Session()
#Once the TensorFlow graph has been created, we have to create a 
#TensorFlow session which is used to execute the graph.

session.run(tf.global_variables_initializer())
#The variables for weights and biases must be initialized before we start optimizing them.

batch_size = 80
#Stochastic Gradient Descent which only uses a small batch of images in each iteration of the optimizer.


feed_dict_test = {x: data.test.images,
                  y_true: data.test.labels}
                  #y_true_cls: data.test.cls}

In [13]:
#Optimize Function

def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = data.train.next_batch(batch_size=batch_size)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch, y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)
        
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [ ]:
#print_accuracy()

In [ ]:
optimize(num_iterations=1)
print_accuracy()

In [ ]:
optimize(num_iterations=1000)
print_accuracy()

In [ ]:
optimize(num_iterations=10000)
print_accuracy()